<a href="https://colab.research.google.com/github/Gonzal22/act-7-/blob/main/act7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
df = pd.read_csv('cuentas_credicel.csv', encoding='latin-1')
df.head()

<ipython-input-3-9bfc5b1acc19>:1: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('cuentas_credicel.csv', encoding='latin-1')


,folio,tag,folio_solicitud,fecha,marca,modelo,plazo,precio,enganche,descuento,...,porc_eng,limite_credito,semana_actual,cp_cliente,edad_cliente,cd_cliente,edo_cliente,cd_venta,edo_venta,curp
0,3,HDNAWQ3A,30,12/11/21 19:00,ZTE,Blade A3 2020 RO B,26S,1949.0,780.0,0.0,...,40.02,5000,26,NaN,23.0,0,0,Tantoyuca,Veracruz,AEVM980413MVZNDR02
1,4,M418YNR4,33,12/11/21 19:15,MOTOROLA,LTE XT2097-12 E7I POWER Azul,13S,2999.0,1050.0,0.0,...,35.01,6000,13,NaN,60.0,PUEBLA,PUE.,Heroica Puebla de Zaragoza,Puebla,ZALC610119MPLPRR07
2,5,SZ7V3NZT,37,13/11/21 14:13,ZTE,BLADE A3 2020 Gris,13S,1959.0,490.0,0.0,...,25.01,0,74,NaN,58.0,TANTOYUCA,VER.,0,0,0
3,6,9I08UIQZ,38,13/11/21 15:45,ZTE,BLADE A3 2020 Gris,13S,1959.0,490.0,0.0,...,25.01,7000,13,NaN,42.0,AMXTLAN PUE,NaN,Zacatlan,Puebla,SAMJ790115MPLNRS02
4,7,JMF4LKYU,44,13/11/21 16:10,MOTOROLA,G10 B,39S,4299.0,1075.0,0.0,...,25.01,7000,74,NaN,33.0,TANTOYUCA,VER.,Tantoyuca,Veracruz,TEHF880707HVZRRR07


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22735 entries, 0 to 22734
Data columns (total 39 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   folio               22735 non-null  int64  
 1   tag                 22735 non-null  object 
 2   folio_solicitud     22735 non-null  int64  
 3   fecha               22735 non-null  object 
 4   marca               22735 non-null  object 
 5   modelo              22735 non-null  object 
 6   plazo               22735 non-null  object 
 7   precio              22735 non-null  float64
 8   enganche            22735 non-null  float64
 9   descuento           22735 non-null  float64
 10  semana              22735 non-null  int64  
 11  monto_financiado    22735 non-null  float64
 12  costo_total         22735 non-null  int64  
 13  monto_accesorios    22735 non-null  float64
 14  agente_venta        22735 non-null  object 
 15  dis_venta           22735 non-null  object 
 16  stat

In [5]:
df.isnull().sum()

folio                    0
tag                      0
folio_solicitud          0
fecha                    0
marca                    0
modelo                   0
plazo                    0
precio                   0
enganche                 0
descuento                0
semana                   0
monto_financiado         0
costo_total              0
monto_accesorios         0
agente_venta             0
dis_venta                0
status                   0
fraude                   0
empresa                 13
inversion                0
pagos_realizados         0
reautorizacion           0
fecha_ultimo_pago     3356
fecha_pago_proximo    3356
status_cuenta         3314
puntos                   0
riesgo                   0
score_buro               0
razones_buro          7340
porc_eng                 0
limite_credito           0
semana_actual            0
cp_cliente            7243
edad_cliente             9
cd_cliente               0
edo_cliente              8
cd_venta                 0
e

In [6]:
df=df.fillna(method="bfill")
df=df.fillna(method="ffill")
df.isnull().sum().sum()

0

In [7]:
y=df
Limite_Superior= y.mean() + 3* y.std()
Limite_Inferior= y.mean() - 3 * y.std()
print("Limite Superior Permitido", Limite_Superior)
print("Limite Inferior Permitido", Limite_Inferior)

<ipython-input-7-3b2f7da30154>:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  Limite_Superior= y.mean() + 3* y.std()
<ipython-input-7-3b2f7da30154>:2: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  Limite_Superior= y.mean() + 3* y.std()


Limite Superior Permitido folio                31323.107698
folio_solicitud     127723.906961
precio                8081.558082
enganche              2356.496845
descuento              478.215020
semana                 541.287392
monto_financiado      6366.317810
costo_total          12857.588246
monto_accesorios       191.260880
status                   2.611178
fraude                   0.864131
inversion                1.511599
pagos_realizados        39.306132
reautorizacion           0.578086
puntos                  48.354233
riesgo                  58.456322
score_buro            1142.021000
porc_eng                45.711306
semana_actual           64.380898
edad_cliente           422.314014
dtype: float64
Limite Inferior Permitido folio               -8301.055796
folio_solicitud    -36023.715538
precio                -87.484900
enganche             -503.622066
descuento            -321.263535
semana                -65.792692
monto_financiado     -214.044583
costo_total         -1

<ipython-input-7-3b2f7da30154>:3: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  Limite_Inferior= y.mean() - 3 * y.std()
<ipython-input-7-3b2f7da30154>:3: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  Limite_Inferior= y.mean() - 3 * y.std()


In [8]:
df2=df.select_dtypes(include=["number"])
df3=df.select_dtypes(include=["object"])

df2

,folio,folio_solicitud,precio,enganche,descuento,semana,monto_financiado,costo_total,monto_accesorios,status,fraude,inversion,pagos_realizados,reautorizacion,puntos,riesgo,score_buro,porc_eng,semana_actual,edad_cliente
0,3,30,1949.0,780.0,0.0,81,1169.0,2106,0.0,2,0,0,0,0,0,0.00,0,40.02,26,23.0
1,4,33,2999.0,1050.0,0.0,225,1949.0,2925,0.0,2,0,0,0,0,0,0.00,0,35.01,13,60.0
2,5,37,1959.0,490.0,0.0,170,1469.0,2210,0.0,3,0,0,0,0,0,0.00,0,25.01,74,58.0
3,6,38,1959.0,490.0,0.0,170,1469.0,2210,0.0,2,0,0,0,0,0,0.00,0,25.01,13,42.0
4,7,44,4299.0,1075.0,0.0,178,3224.0,6942,0.0,1,0,0,0,0,0,0.00,0,25.01,74,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22730,22967,96502,3349.0,297.0,0.0,211,3052.0,5486,0.0,1,0,0,0,0,10,27.40,469,8.87,0,22.0
22731,22968,96479,4100.0,1209.0,0.0,200,2891.0,5200,0.0,1,0,0,0,0,9,19.00,502,29.49,0,27.0
22732,22969,96442,4849.0,1152.0,0.0,256,3697.0,6656,0.0,1,0,0,0,0,11,26.65,459,23.76,0,36.0
22733,22970,96536,9799.0,2799.0,0.0,386,7000.0,15054,0.0,1,0,0,0,0,14,-17.35,699,28.56,0,53.0


In [9]:
df2= df[(y<=Limite_Superior)&(y>=Limite_Inferior)]
df2.head(20)

<ipython-input-9-6e8bf59fb3a1>:1: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version. Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  df2= df[(y<=Limite_Superior)&(y>=Limite_Inferior)]


,folio,tag,folio_solicitud,fecha,marca,modelo,plazo,precio,enganche,descuento,...,porc_eng,limite_credito,semana_actual,cp_cliente,edad_cliente,cd_cliente,edo_cliente,cd_venta,edo_venta,curp
0,3,NaN,30,NaN,NaN,NaN,NaN,1949.0,780.0,0.0,...,40.02,NaN,26.0,NaN,23.0,NaN,NaN,NaN,NaN,NaN
1,4,NaN,33,NaN,NaN,NaN,NaN,2999.0,1050.0,0.0,...,35.01,NaN,13.0,NaN,60.0,NaN,NaN,NaN,NaN,NaN
2,5,NaN,37,NaN,NaN,NaN,NaN,1959.0,490.0,0.0,...,25.01,NaN,NaN,NaN,58.0,NaN,NaN,NaN,NaN,NaN
3,6,NaN,38,NaN,NaN,NaN,NaN,1959.0,490.0,0.0,...,25.01,NaN,13.0,NaN,42.0,NaN,NaN,NaN,NaN,NaN
4,7,NaN,44,NaN,NaN,NaN,NaN,4299.0,1075.0,0.0,...,25.01,NaN,NaN,NaN,33.0,NaN,NaN,NaN,NaN,NaN
5,8,NaN,49,NaN,NaN,NaN,NaN,2999.0,750.0,0.0,...,25.01,NaN,26.0,NaN,38.0,NaN,NaN,NaN,NaN,NaN
6,9,NaN,50,NaN,NaN,NaN,NaN,2999.0,900.0,0.0,...,30.01,NaN,26.0,NaN,21.0,NaN,NaN,NaN,NaN,NaN
7,10,NaN,54,NaN,NaN,NaN,NaN,3999.0,1500.0,0.0,...,37.51,NaN,13.0,NaN,45.0,NaN,NaN,NaN,NaN,NaN
8,11,NaN,59,NaN,NaN,NaN,NaN,2889.0,1000.0,0.0,...,34.61,NaN,26.0,NaN,36.0,NaN,NaN,NaN,NaN,NaN
9,12,NaN,62,NaN,NaN,NaN,NaN,2999.0,750.0,0.0,...,25.01,NaN,NaN,NaN,47.0,NaN,NaN,NaN,NaN,NaN


In [10]:
df2=df.select_dtypes(include=["number"])
df2 = df2.fillna(round(df.mean(),1))
df2

<ipython-input-10-143e3a29e78f>:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df2 = df2.fillna(round(df.mean(),1))


,folio,folio_solicitud,precio,enganche,descuento,semana,monto_financiado,costo_total,monto_accesorios,status,fraude,inversion,pagos_realizados,reautorizacion,puntos,riesgo,score_buro,porc_eng,semana_actual,edad_cliente
0,3,30,1949.0,780.0,0.0,81,1169.0,2106,0.0,2,0,0,0,0,0,0.00,0,40.02,26,23.0
1,4,33,2999.0,1050.0,0.0,225,1949.0,2925,0.0,2,0,0,0,0,0,0.00,0,35.01,13,60.0
2,5,37,1959.0,490.0,0.0,170,1469.0,2210,0.0,3,0,0,0,0,0,0.00,0,25.01,74,58.0
3,6,38,1959.0,490.0,0.0,170,1469.0,2210,0.0,2,0,0,0,0,0,0.00,0,25.01,13,42.0
4,7,44,4299.0,1075.0,0.0,178,3224.0,6942,0.0,1,0,0,0,0,0,0.00,0,25.01,74,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22730,22967,96502,3349.0,297.0,0.0,211,3052.0,5486,0.0,1,0,0,0,0,10,27.40,469,8.87,0,22.0
22731,22968,96479,4100.0,1209.0,0.0,200,2891.0,5200,0.0,1,0,0,0,0,9,19.00,502,29.49,0,27.0
22732,22969,96442,4849.0,1152.0,0.0,256,3697.0,6656,0.0,1,0,0,0,0,11,26.65,459,23.76,0,36.0
22733,22970,96536,9799.0,2799.0,0.0,386,7000.0,15054,0.0,1,0,0,0,0,14,-17.35,699,28.56,0,53.0


In [11]:
final= pd.concat([df2, df3], axis = 1)
final.head()

,folio,folio_solicitud,precio,enganche,descuento,semana,monto_financiado,costo_total,monto_accesorios,status,...,fecha_pago_proximo,status_cuenta,razones_buro,limite_credito,cp_cliente,cd_cliente,edo_cliente,cd_venta,edo_venta,curp
0,3,30,1949.0,780.0,0.0,81,1169.0,2106,0.0,2,...,12/03/22 0:00,Cancelado,C1,5000,72490,0,0,Tantoyuca,Veracruz,AEVM980413MVZNDR02
1,4,33,2999.0,1050.0,0.0,225,1949.0,2925,0.0,2,...,12/03/22 0:00,Cancelado,C1,6000,72490,PUEBLA,PUE.,Heroica Puebla de Zaragoza,Puebla,ZALC610119MPLPRR07
2,5,37,1959.0,490.0,0.0,170,1469.0,2210,0.0,3,...,12/03/22 0:00,Cancelado,C1,0,72490,TANTOYUCA,VER.,0,0,0
3,6,38,1959.0,490.0,0.0,170,1469.0,2210,0.0,2,...,12/03/22 0:00,Mora,C1,7000,72490,AMXTLAN PUE,VER.,Zacatlan,Puebla,SAMJ790115MPLNRS02
4,7,44,4299.0,1075.0,0.0,178,3224.0,6942,0.0,1,...,12/03/22 0:00,Mora,C1,7000,72490,TANTOYUCA,VER.,Tantoyuca,Veracruz,TEHF880707HVZRRR07


In [21]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22735 entries, 0 to 22734
Data columns (total 39 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   folio               22735 non-null  int64  
 1   folio_solicitud     22735 non-null  int64  
 2   precio              22735 non-null  float64
 3   enganche            22735 non-null  float64
 4   descuento           22735 non-null  float64
 5   semana              22735 non-null  int64  
 6   monto_financiado    22735 non-null  float64
 7   costo_total         22735 non-null  int64  
 8   monto_accesorios    22735 non-null  float64
 9   status              22735 non-null  int64  
 10  fraude              22735 non-null  int64  
 11  inversion           22735 non-null  int64  
 12  pagos_realizados    22735 non-null  int64  
 13  reautorizacion      22735 non-null  int64  
 14  puntos              22735 non-null  int64  
 15  riesgo              22735 non-null  float64
 16  scor

In [12]:
final.isnull().sum().sum()

0

#**1**

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
Vars_Indep= final[['precio', 'enganche', 'descuento']]
Var_Dep= final['marca']
X= Vars_Indep
y= Var_Dep
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)
escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)
y_pred = algoritmo.predict(X_test)
y_pred
matriz = confusion_matrix(y_test, y_pred)
exactitud = accuracy_score(y_test, y_pred)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [32]:
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
print("matriz: ",matriz)
precision = precision_score(y_test, y_pred, average="macro")
print("precision:",precision)
sensibilidad = recall_score(y_test, y_pred, average="macro")
print("sensibilidad:",sensibilidad)
puntajef1 = f1_score(y_test, y_pred, average="macro")
print("F1:",puntajef1)
print("exactitud:",exactitud)

matriz:  [[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    2    0   13    0    0    1    0    0    0   35]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0   12    0    0    0    0    0    0   12]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    1]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    1    0   35    0    0    1    0    0    0   41]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    2    0   81    0    0    3    0    0    0   22]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    1    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    1]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


sensibilidad: 0.07668962622341213
F1: 0.0638439033871148
exactitud: 0.45755754288227535


#**2**

In [37]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
Vars_Indep= final[['puntos', 'inversion', 'precio']]
Var_Dep= final['status_cuenta']
X= Vars_Indep
y= Var_Dep
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)
escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)
y_pred = algoritmo.predict(X_test)
y_pred
matriz = confusion_matrix(y_test, y_pred)
exactitud = accuracy_score(y_test, y_pred)

In [38]:
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
print("matriz: ",matriz)
precision = precision_score(y_test, y_pred, average="macro")
print("precision:",precision)
sensibilidad = recall_score(y_test, y_pred, average="macro")
print("sensibilidad:",sensibilidad)
puntajef1 = f1_score(y_test, y_pred, average="macro")
print("F1:",puntajef1)
print("exactitud:",exactitud)

matriz:  [[   0    0  969    0   24]
 [   0    0   78    0    0]
 [   0    0 3287    0   30]
 [   0    0  627    0   13]
 [   0    0 1721    0   72]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


precision: 0.20198083975202358
sensibilidad: 0.2062223691402989
F1: 0.14639998161306814
exactitud: 0.4924497874211992


#**3**

In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
Vars_Indep= final[['puntos', 'porc_eng', 'riesgo']]
Var_Dep= final['inversion']
X= Vars_Indep
y= Var_Dep
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)
escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)
y_pred = algoritmo.predict(X_test)
y_pred
matriz = confusion_matrix(y_test, y_pred)
exactitud = accuracy_score(y_test, y_pred)

In [40]:
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
print("matriz: ",matriz)
precision = precision_score(y_test, y_pred, average="macro")
print("precision:",precision)
sensibilidad = recall_score(y_test, y_pred, average="macro")
print("sensibilidad:",sensibilidad)
puntajef1 = f1_score(y_test, y_pred, average="macro")
print("F1:",puntajef1)
print("exactitud:",exactitud)

matriz:  [[5193    0]
 [1628    0]]
precision: 0.38066265943410055
sensibilidad: 0.5
F1: 0.43224571333444317
exactitud: 0.7613253188682011


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#**4**

In [41]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
Vars_Indep= final[['precio', 'enganche', 'inversion']]
Var_Dep= final['plazo']
X= Vars_Indep
y= Var_Dep
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)
escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)
y_pred = algoritmo.predict(X_test)
y_pred
matriz = confusion_matrix(y_test, y_pred)
exactitud = accuracy_score(y_test, y_pred)

In [42]:
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
print("matriz: ",matriz)
precision = precision_score(y_test, y_pred, average="macro")
print("precision:",precision)
sensibilidad = recall_score(y_test, y_pred, average="macro")
print("sensibilidad:",sensibilidad)
puntajef1 = f1_score(y_test, y_pred, average="macro")
print("F1:",puntajef1)
print("exactitud:",exactitud)

matriz:  [[  74 1728   60    0]
 [  35 3043  124    0]
 [   1 1474  149    0]
 [   2  116   15    0]]
precision: 0.39181477678918303


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


sensibilidad: 0.2704586291094733
F1: 0.21562536435602753
exactitud: 0.4788154229585105


#**5**

In [43]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
Vars_Indep= final[['status', 'puntos', 'riesgo']]
Var_Dep= final['razones_buro']
X= Vars_Indep
y= Var_Dep
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)
escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)
y_pred = algoritmo.predict(X_test)
y_pred
matriz = confusion_matrix(y_test, y_pred)
exactitud = accuracy_score(y_test, y_pred)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [44]:
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
print("matriz: ",matriz)
precision = precision_score(y_test, y_pred, average="macro")
print("precision:",precision)
sensibilidad = recall_score(y_test, y_pred, average="macro")
print("sensibilidad:",sensibilidad)
puntajef1 = f1_score(y_test, y_pred, average="macro")
print("F1:",puntajef1)
print("exactitud:",exactitud)

matriz:  [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
precision: 0.0018447624983672585
sensibilidad: 0.0035692495870746585
F1: 0.002320173538444939
exactitud: 0.3267849288960563


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#**6**

In [45]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
Vars_Indep= final[['precio', 'descuento', 'semana']]
Var_Dep= final['dis_venta']
X= Vars_Indep
y= Var_Dep
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)
escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)
y_pred = algoritmo.predict(X_test)
y_pred
matriz = confusion_matrix(y_test, y_pred)
exactitud = accuracy_score(y_test, y_pred)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [46]:
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
print("matriz: ",matriz)
precision = precision_score(y_test, y_pred, average="macro")
print("precision:",precision)
sensibilidad = recall_score(y_test, y_pred, average="macro")
print("sensibilidad:",sensibilidad)
puntajef1 = f1_score(y_test, y_pred, average="macro")
print("F1:",puntajef1)
print("exactitud:",exactitud)

matriz:  [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
precision: 0.0013141396707147775
sensibilidad: 0.005918147381880437
F1: 0.001653836443235518
exactitud: 0.04090309338806627


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#**7**

In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
Vars_Indep= final[['monto_financiado', 'monto_accesorios', 'precio']]
Var_Dep= final['empresa']
X= Vars_Indep
y= Var_Dep
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)
escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)
y_pred = algoritmo.predict(X_test)
y_pred
matriz = confusion_matrix(y_test, y_pred)
exactitud = accuracy_score(y_test, y_pred)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [48]:
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
print("matriz: ",matriz)
precision = precision_score(y_test, y_pred, average="macro")
print("precision:",precision)
sensibilidad = recall_score(y_test, y_pred, average="macro")
print("sensibilidad:",sensibilidad)
puntajef1 = f1_score(y_test, y_pred, average="macro")
print("F1:",puntajef1)
print("exactitud:",exactitud)

matriz:  [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
precision: 0.034870907186419865
sensibilidad: 0.030116245949390684
F1: 0.02655046051105772
exactitud: 0.4849728778771441


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#**8**

In [49]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
Vars_Indep= final[['precio', 'inversion', 'costo_total']]
Var_Dep= final['cd_cliente']
X= Vars_Indep
y= Var_Dep
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)
escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)
y_pred = algoritmo.predict(X_test)
y_pred
matriz = confusion_matrix(y_test, y_pred)
exactitud = accuracy_score(y_test, y_pred)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [50]:
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
print("matriz: ",matriz)
precision = precision_score(y_test, y_pred, average="macro")
print("precision:",precision)
sensibilidad = recall_score(y_test, y_pred, average="macro")
print("sensibilidad:",sensibilidad)
puntajef1 = f1_score(y_test, y_pred, average="macro")
print("F1:",puntajef1)
print("exactitud:",exactitud)

matriz:  [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
precision: 0.0004094293820039427
sensibilidad: 0.002324271314783938
F1: 0.0006729980208428084
exactitud: 0.11420612813370473


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#**9**

In [51]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
Vars_Indep= final[['inversion', 'costo_total', 'precio']]
Var_Dep= final['cd_venta']
X= Vars_Indep
y= Var_Dep
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)
escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)
y_pred = algoritmo.predict(X_test)
y_pred
matriz = confusion_matrix(y_test, y_pred)
exactitud = accuracy_score(y_test, y_pred)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [52]:
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
print("matriz: ",matriz)
precision = precision_score(y_test, y_pred, average="macro")
print("precision:",precision)
sensibilidad = recall_score(y_test, y_pred, average="macro")
print("sensibilidad:",sensibilidad)
puntajef1 = f1_score(y_test, y_pred, average="macro")
print("F1:",puntajef1)
print("exactitud:",exactitud)

matriz:  [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
precision: 0.0022231694400245893


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


sensibilidad: 0.010236768468192037
F1: 0.00360145277153327
exactitud: 0.1278404925963935


#**10**

In [53]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
Vars_Indep= final[['puntos', 'enganche', 'inversion']]
Var_Dep= final['cp_cliente']
X= Vars_Indep
y= Var_Dep
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)
escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)
y_pred = algoritmo.predict(X_test)
y_pred
matriz = confusion_matrix(y_test, y_pred)
exactitud = accuracy_score(y_test, y_pred)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [54]:
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
print("matriz: ",matriz)
precision = precision_score(y_test, y_pred, average="macro")
print("precision:",precision)
sensibilidad = recall_score(y_test, y_pred, average="macro")
print("sensibilidad:",sensibilidad)
puntajef1 = f1_score(y_test, y_pred, average="macro")
print("F1:",puntajef1)
print("exactitud:",exactitud)

matriz:  [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
precision: 0.0005142491457251431
sensibilidad: 0.0011037252166147785
F1: 0.0006633446100208859
exactitud: 0.26433074329277234


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
